# Basic Species Richness 

In [6]:
#BASCIC SPECIES RICHNESS
#newvalue_future and _historical and sumbin_future and _historical
import xarray as xr
import numpy as np
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

models = ["GAM", "GBM"]
taxas = ["Bird", "Mammals", "Amphibians"]
scenarios = ["rcp26", "rcp60"]
historical_time = 1146
future_time = 65

year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

for time in [historical_time, future_time]:
    for scenario in scenarios:
        # Create empty arrays for storing the results for each taxa separately
        diff_value_bin_combined_per_taxa = {}
        diff_sum_bin_combined_per_taxa = {}
        diff_combined_per_taxa = {}

        for taxa in taxas:
            diff_value_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_sum_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_combined_per_taxa[taxa] = np.zeros([360, 720])

        # Initialize the combined arrays outside the taxa loop
        diff_value_bin_combined = np.zeros([360, 720])
        diff_sum_bin_combined = np.zeros([360, 720])
        diff_combined = np.zeros([360, 720])

        for row, taxa in enumerate(taxas):
            # Reset the per model arrays for each taxa
            diff_value_bin_models = []
            diff_sum_bin_models = []
            diff_models = []
            percent_changes = []

            for model in models:
                newvalue_path = f"/storage/scratch/users/ch21o450/data/SR/{taxa}_{model}_{time}_{scenario}_summedprobs_newvalue.nc"
                sumbin_path = f"/storage/scratch/users/ch21o450/data/SR/{taxa}_{model}_{time}_{scenario}_summedprobs_sum.nc"

                mean_value_bin = xr.open_dataset(newvalue_path, decode_times=False).to_array().isel(variable=0)
                mean_sum_bin = xr.open_dataset(sumbin_path, decode_times=False).to_array().isel(variable=0)

                # Calculate the differences
                diff_value_bin = mean_value_bin
                diff_sum_bin = mean_sum_bin
                diff = diff_sum_bin - diff_value_bin
                change_percent = ((diff_sum_bin - diff_value_bin) / (diff_value_bin)) * 100
                percent_changes.append(change_percent)

                # append the differences to the model-specific lists
                diff_value_bin_models.append(diff_value_bin)
                diff_sum_bin_models.append(diff_sum_bin)
                diff_models.append(diff)

            # Calculate the ensemble mean outside of the innermost loop
            diff_value_bin_ensemble = xr.concat(diff_value_bin_models, dim="ensemble").mean(dim="ensemble")
            diff_sum_bin_ensemble = xr.concat(diff_sum_bin_models, dim="ensemble").mean(dim="ensemble")

            diff_value_bin_ensemble = diff_value_bin_ensemble
            diff_sum_bin_ensemble = diff_sum_bin_ensemble
            diff_ensemble = diff_sum_bin_ensemble - diff_value_bin_ensemble

            # Append the ensemble mean to the overall results for this taxa
            diff_value_bin_combined_per_taxa[taxa] = diff_value_bin_ensemble
            diff_sum_bin_combined_per_taxa[taxa] = diff_sum_bin_ensemble
            diff_combined_per_taxa[taxa] = diff_ensemble

        # Calculate the overall species richness sum over all taxa
        diff_value_bin_combined = sum(diff_value_bin_combined_per_taxa.values())
        diff_sum_bin_combined = sum(diff_sum_bin_combined_per_taxa.values())
        diff_combined = sum(diff_combined_per_taxa.values())

        diff_newvalue = diff_value_bin_combined
        diff_sumbin = diff_sum_bin_combined
        diff_combined = diff_combined

        # Get rid of fine line at equator
        land_sea_mask = xr.open_dataset("/storage/workspaces/wa_climate/climate_trt/chari/LUH2/remapped_luh2_ssp126.nc", decode_times=False).primf
        diff_newvalue_masked = diff_newvalue.where(land_sea_mask >= 0)
        diff_newvalue = diff_newvalue_masked.isel(time=0)

        output_path_diff_newvalue = f"/storage/scratch/users/ch21o450/data/SRnew/SR_CC_{time}_{scenario}.nc"
        diff_newvalue.to_netcdf(output_path_diff_newvalue, format='NETCDF4')

        # Write out diff_sumbin as netCDF
        output_path_diff_sumbin = f"/storage/scratch/users/ch21o450/data/SRnew/SR_CCLUC_{time}_{scenario}.nc"
        diff_sumbin.to_netcdf(output_path_diff_sumbin, format='NETCDF4')


per SDM and GCM

In [1]:
#BASCIC SPECIES RICHNESS
#newvalue_future and _historical and sumbin_future and _historical
import xarray as xr
import numpy as np
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

models = ["GAM", "GBM"]
taxas = ["Bird", "Mammals", "Amphibians"]
scenarios = ["rcp26", "rcp60"]
historical_time = 1146
future_time = 65

model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']

year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

for time in [historical_time, future_time]:
    for scenario in scenarios:
        # Create empty arrays for storing the results for each taxa separately
        diff_value_bin_combined_per_taxa = {}
        diff_sum_bin_combined_per_taxa = {}
        diff_combined_per_taxa = {}

        for taxa in taxas:
            diff_value_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_sum_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_combined_per_taxa[taxa] = np.zeros([360, 720])

        # Initialize the combined arrays outside the taxa loop
        diff_value_bin_combined = np.zeros([360, 720])
        diff_sum_bin_combined = np.zeros([360, 720])
        diff_combined = np.zeros([360, 720])

        for row, taxa in enumerate(taxas):
            # Reset the per model arrays for each taxa
            diff_value_bin_models = []
            diff_sum_bin_models = []
            diff_models = []
            percent_changes = []

            for model in models:
                for model_name in model_names:
                    newvalue_path = f"/storage/scratch/users/ch21o450/data/SR/{taxa}_{model}_{model_name}_{time}_{scenario}_summedprobs_newvalue.nc"
                    sumbin_path = f"/storage/scratch/users/ch21o450/data/SR/{taxa}_{model}_{model_name}_{time}_{scenario}_summedprobs_sum.nc"

                    mean_value_bin = xr.open_dataset(newvalue_path, decode_times=False).to_array().isel(variable=0)
                    mean_sum_bin = xr.open_dataset(sumbin_path, decode_times=False).to_array().isel(variable=0)

                    # Calculate the differences
                    diff_value_bin = mean_value_bin
                    diff_sum_bin = mean_sum_bin
                    diff = diff_sum_bin - diff_value_bin
                    change_percent = ((diff_sum_bin - diff_value_bin) / (diff_value_bin)) * 100
                    percent_changes.append(change_percent)


                    # Append the ensemble mean to the overall results for this taxa
                    diff_value_bin_combined_per_taxa[taxa] = diff_value_bin
                    diff_sum_bin_combined_per_taxa[taxa] = diff_sum_bin
                    diff_combined_per_taxa[taxa] = diff_ensemble

                    # Calculate the overall species richness sum over all taxa
                    diff_value_bin_combined = sum(diff_value_bin_combined_per_taxa.values())
                    diff_sum_bin_combined = sum(diff_sum_bin_combined_per_taxa.values())
                    diff_combined = sum(diff_combined_per_taxa.values())

                    diff_newvalue = diff_value_bin_combined
                    diff_sumbin = diff_sum_bin_combined
                    diff_combined = diff_combined

                    # Get rid of fine line at equator
                    land_sea_mask = xr.open_dataset("/storage/workspaces/wa_climate/climate_trt/chari/LUH2/remapped_luh2_ssp126.nc", decode_times=False).primf
                    diff_newvalue_masked = diff_newvalue.where(land_sea_mask >= 0)
                    diff_newvalue = diff_newvalue_masked.isel(time=0)

                    output_path_diff_newvalue = f"/storage/scratch/users/ch21o450/data/SRind/SR_CC_{time}_{scenario}_{model}_{model_name}.nc"
                    diff_newvalue.to_netcdf(output_path_diff_newvalue, format='NETCDF4')

                    # Write out diff_sumbin as netCDF
                    output_path_diff_sumbin = f"/storage/scratch/users/ch21o450/data/SRind/SR_CCLUC_{time}_{scenario}_{model}_{model_name}.nc"
                    diff_sumbin.to_netcdf(output_path_diff_sumbin, format='NETCDF4')


FileNotFoundError: [Errno 2] No such file or directory: b'/storage/scratch/users/ch21o450/data/SR/Bird_GAM_GFDL-ESM2M_1146_rcp26_summedprobs_newvalue.nc'

# Sensitivity Analysis

In [1]:
#Sensitivity Analysis SPECIES RICHNESS
#newvalue_future and _historical and sumbin_future and _historical
import xarray as xr
import numpy as np
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

models = ["GAM", "GBM"]
taxas = ["Bird", "Mammals", "Amphibians"]
scenarios = ["rcp26", "rcp60"]
historical_time = 1146
future_time = 65

year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

for time in [historical_time, future_time]:
    for scenario in scenarios:
        # Create empty arrays for storing the results for each taxa separately
        diff_value_bin_combined_per_taxa = {}
        diff_sum_bin_combined_per_taxa = {}
        diff_combined_per_taxa = {}

        for taxa in taxas:
            diff_value_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_sum_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_combined_per_taxa[taxa] = np.zeros([360, 720])

        # Initialize the combined arrays outside the taxa loop
        diff_value_bin_combined = np.zeros([360, 720])
        diff_sum_bin_combined = np.zeros([360, 720])
        diff_combined = np.zeros([360, 720])

        for row, taxa in enumerate(taxas):
            # Reset the per model arrays for each taxa
            diff_value_bin_models = []
            diff_sum_bin_models = []
            diff_models = []
            percent_changes = []

            for model in models:
                newvalue_path = f"/storage/scratch/users/ch21o450/data/SR/SA_{taxa}_{model}_{time}_{scenario}_summedprobs_newvalue.nc"
                sumbin_path = f"/storage/scratch/users/ch21o450/data/SR/SA_{taxa}_{model}_{time}_{scenario}_summedprobs_sum.nc"

                mean_value_bin = xr.open_dataset(newvalue_path, decode_times=False).to_array().isel(variable=0)
                mean_sum_bin = xr.open_dataset(sumbin_path, decode_times=False).to_array().isel(variable=0)

                # Calculate the differences
                diff_value_bin = mean_value_bin
                diff_sum_bin = mean_sum_bin
                diff = diff_sum_bin - diff_value_bin
                change_percent = ((diff_sum_bin - diff_value_bin) / (diff_value_bin)) * 100
                percent_changes.append(change_percent)

                # append the differences to the model-specific lists
                diff_value_bin_models.append(diff_value_bin)
                diff_sum_bin_models.append(diff_sum_bin)
                diff_models.append(diff)

            # Calculate the ensemble mean outside of the innermost loop
            diff_value_bin_ensemble = xr.concat(diff_value_bin_models, dim="ensemble").mean(dim="ensemble")
            diff_sum_bin_ensemble = xr.concat(diff_sum_bin_models, dim="ensemble").mean(dim="ensemble")

            diff_value_bin_ensemble = diff_value_bin_ensemble
            diff_sum_bin_ensemble = diff_sum_bin_ensemble
            diff_ensemble = diff_sum_bin_ensemble - diff_value_bin_ensemble

            # Append the ensemble mean to the overall results for this taxa
            diff_value_bin_combined_per_taxa[taxa] = diff_value_bin_ensemble
            diff_sum_bin_combined_per_taxa[taxa] = diff_sum_bin_ensemble
            diff_combined_per_taxa[taxa] = diff_ensemble

        # Calculate the overall species richness sum over all taxa
        diff_value_bin_combined = sum(diff_value_bin_combined_per_taxa.values())
        diff_sum_bin_combined = sum(diff_sum_bin_combined_per_taxa.values())
        diff_combined = sum(diff_combined_per_taxa.values())

        diff_newvalue = diff_value_bin_combined
        diff_sumbin = diff_sum_bin_combined
        diff_combined = diff_combined

        # Get rid of fine line at equator
        land_sea_mask = xr.open_dataset("/storage/workspaces/wa_climate/climate_trt/chari/LUH2/remapped_luh2_ssp126.nc", decode_times=False).primf
        diff_newvalue_masked = diff_newvalue.where(land_sea_mask >= 0)
        diff_newvalue = diff_newvalue_masked.isel(time=0)

        output_path_diff_newvalue = f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{time}_{scenario}_SA.nc"
        diff_newvalue.to_netcdf(output_path_diff_newvalue, format='NETCDF4')

        # Write out diff_sumbin as netCDF
        output_path_diff_sumbin = f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{time}_{scenario}_SA.nc"
        diff_sumbin.to_netcdf(output_path_diff_sumbin, format='NETCDF4')


# Dispersal sceenario 2

In [2]:
#Dispersal scenario
#newvalue_future and _historical and sumbin_future and _historical
import xarray as xr
import numpy as np
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

models = ["GAM", "GBM"]
taxas = ["Bird", "Mammals", "Amphibians"]
scenarios = ["rcp26", "rcp60"]
#historical_time = 1146
future_time = 1146

year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

for time in [future_time]:
    for scenario in scenarios:
        # Create empty arrays for storing the results for each taxa separately
        diff_value_bin_combined_per_taxa = {}
        diff_sum_bin_combined_per_taxa = {}
        diff_combined_per_taxa = {}

        for taxa in taxas:
            diff_value_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_sum_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_combined_per_taxa[taxa] = np.zeros([360, 720])

        # Initialize the combined arrays outside the taxa loop
        diff_value_bin_combined = np.zeros([360, 720])
        diff_sum_bin_combined = np.zeros([360, 720])
        diff_combined = np.zeros([360, 720])

        for row, taxa in enumerate(taxas):
            # Reset the per model arrays for each taxa
            diff_value_bin_models = []
            diff_sum_bin_models = []
            diff_models = []
            percent_changes = []

            for model in models:
                newvalue_path = f"/storage/scratch/users/ch21o450/data/SR/Dispersal2_{taxa}_{model}_{time}_{scenario}_summedprobs_newvalue.nc"
                sumbin_path = f"/storage/scratch/users/ch21o450/data/SR/Dispersal2_{taxa}_{model}_{time}_{scenario}_summedprobs_sum.nc"

                mean_value_bin = xr.open_dataset(newvalue_path, decode_times=False).to_array().isel(variable=0)
                mean_sum_bin = xr.open_dataset(sumbin_path, decode_times=False).to_array().isel(variable=0)

                # Calculate the differences
                diff_value_bin = mean_value_bin
                diff_sum_bin = mean_sum_bin
                diff = diff_sum_bin - diff_value_bin
                change_percent = ((diff_sum_bin - diff_value_bin) / (diff_value_bin)) * 100
                percent_changes.append(change_percent)

                # append the differences to the model-specific lists
                diff_value_bin_models.append(diff_value_bin)
                diff_sum_bin_models.append(diff_sum_bin)
                diff_models.append(diff)

            # Calculate the ensemble mean outside of the innermost loop
            diff_value_bin_ensemble = xr.concat(diff_value_bin_models, dim="ensemble").mean(dim="ensemble")
            diff_sum_bin_ensemble = xr.concat(diff_sum_bin_models, dim="ensemble").mean(dim="ensemble")

            diff_value_bin_ensemble = diff_value_bin_ensemble
            diff_sum_bin_ensemble = diff_sum_bin_ensemble
            diff_ensemble = diff_sum_bin_ensemble - diff_value_bin_ensemble

            # Append the ensemble mean to the overall results for this taxa
            diff_value_bin_combined_per_taxa[taxa] = diff_value_bin_ensemble
            diff_sum_bin_combined_per_taxa[taxa] = diff_sum_bin_ensemble
            diff_combined_per_taxa[taxa] = diff_ensemble

        # Calculate the overall species richness sum over all taxa
        diff_value_bin_combined = sum(diff_value_bin_combined_per_taxa.values())
        diff_sum_bin_combined = sum(diff_sum_bin_combined_per_taxa.values())
        diff_combined = sum(diff_combined_per_taxa.values())

        diff_newvalue = diff_value_bin_combined
        diff_sumbin = diff_sum_bin_combined
        diff_combined = diff_combined

        # Get rid of fine line at equator
        land_sea_mask = xr.open_dataset("/storage/workspaces/wa_climate/climate_trt/chari/LUH2/remapped_luh2_ssp126.nc", decode_times=False).primf
        diff_newvalue_masked = diff_newvalue.where(land_sea_mask >= 0)
        diff_newvalue = diff_newvalue_masked.isel(time=0)

        output_path_diff_newvalue = f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{time}_{scenario}_dispersal2.nc"
        diff_newvalue.to_netcdf(output_path_diff_newvalue, format='NETCDF4')

        # Write out diff_sumbin as netCDF
        output_path_diff_sumbin = f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{time}_{scenario}_dispersal2.nc"
        diff_sumbin.to_netcdf(output_path_diff_sumbin, format='NETCDF4')


# Future only SR

In [8]:
### write out cc anc luc (future only) per scenario

warnings.filterwarnings('ignore', category=UserWarning)

models =  ["GAM","GBM"]
taxas = ["Bird","Mammals","Amphibians"]
scenarios = ["rcp26"]
historical_time= 1146
future_time=85

model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050','2052', '2056', '2080', '2100', '2150', '2200', '2250']

year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}
# Create empty arrays for storing the results for each taxa separately
diff_value_bin_combined_per_taxa = {}
diff_sum_bin_combined_per_taxa = {}
diff_combined_per_taxa = {}

for taxa in taxas:

    diff_value_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
    diff_sum_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
    diff_combined_per_taxa[taxa] = np.zeros([360, 720])

for row, scenario in enumerate(scenarios):

    # Initialize the combined arrays outside the taxa loop
    diff_value_bin_combined = np.zeros([360, 720])
    diff_sum_bin_combined = np.zeros([360, 720])
    diff_combined = np.zeros([360, 720])

    for taxa in taxas:
        # Reset the per model arrays for each taxa
        diff_value_bin_models = []
        diff_sum_bin_models = []
        diff_models = []
        percent_changes = []


        for model in models:
            newvalue_hist = f"/storage/scratch/users/ch21o450/data/SR/{taxa}_{model}_{historical_time}_{scenario}_summedprobs_newvalue.nc"
            sumbin_hist = f"/storage/scratch/users/ch21o450/data/SR/{taxa}_{model}_{historical_time}_{scenario}_summedprobs_sum.nc"


            mean_value_bin_hist = xr.open_dataset(newvalue_hist, decode_times=False).to_array().isel(variable=0)
            mean_sum_bin_hist = xr.open_dataset(sumbin_hist, decode_times=False).to_array().isel(variable=0)

            newvalue_fut = f"/storage/scratch/users/ch21o450/data/SR/{taxa}_{model}_{future_time}_{scenario}_summedprobs_newvalue.nc"
            sumbin_fut = f"/storage/scratch/users/ch21o450/data/SR/{taxa}_{model}_{future_time}_{scenario}_summedprobs_sum.nc"


            mean_value_bin_future = xr.open_dataset(newvalue_fut, decode_times=False).to_array()
            mean_sum_bin_future = xr.open_dataset(sumbin_fut, decode_times=False).to_array()
            mean_sum_bin_future = mean_sum_bin_future.isel(variable=0)

            # Calculate the differences
            diff_value_bin = mean_value_bin_future
            diff_sum_bin = mean_sum_bin_future
            diff = diff_sum_bin - diff_value_bin
            change_percent = ((diff_sum_bin - diff_value_bin) / (diff_value_bin)) * 100  # added small value to avoid division by zero
            percent_changes.append(change_percent)

            # append the differences to the model-specific lists
            diff_value_bin_models.append(diff_value_bin)
            diff_sum_bin_models.append(diff_sum_bin)
            diff_models.append(diff)

# Calculate the ensemble mean outside of the innermost loop
        diff_value_bin_ensemble = xr.concat(diff_value_bin_models, dim="ensemble").mean(dim="ensemble")
        diff_sum_bin_ensemble = xr.concat(diff_sum_bin_models, dim="ensemble").mean(dim="ensemble")

        diff_value_bin_ensemble = diff_value_bin_ensemble.isel(variable=0)
        diff_sum_bin_ensemble = diff_sum_bin_ensemble
        diff_ensemble = diff_sum_bin_ensemble - diff_value_bin_ensemble

        # Append the ensemble mean to the overall results for this taxa
        diff_value_bin_combined_per_taxa[taxa] = diff_value_bin_ensemble
        diff_sum_bin_combined_per_taxa[taxa] = diff_sum_bin_ensemble
        diff_combined_per_taxa[taxa] = diff_ensemble

    # Calculate the overall species richness sum over all taxa
    diff_value_bin_combined = sum(diff_value_bin_combined_per_taxa.values())
    diff_sum_bin_combined = sum(diff_sum_bin_combined_per_taxa.values())
    diff_combined = sum(diff_combined_per_taxa.values())



    diff_newvalue= diff_value_bin_combined
    diff_sumbin = diff_sum_bin_combined
    diff = diff_combined

    #Get rid off fine line at equator
    land_sea_mask = xr.open_dataset("/storage/workspaces/wa_climate/climate_trt/chari/LUH2/remapped_luh2_ssp126.nc", decode_times=False).primf
    diff_newvalue_masked = diff_newvalue.where(land_sea_mask >= 0)
    diff_newvalue = diff_newvalue_masked.isel(time=0)


    output_path_diff_newvalue = f"/storage/scratch/users/ch21o450/data/SR/cc_{future_time}_{scenario}.nc"
    diff_newvalue.to_netcdf(output_path_diff_newvalue, format='NETCDF4')

    # Write out diff_sumbin as netCDF
    output_path_diff_sumbin = f"/storage/scratch/users/ch21o450/data/SR/ccluc_{future_time}_{scenario}.nc"
    diff_sumbin.to_netcdf(output_path_diff_sumbin, format='NETCDF4')

# Protected Areas

In [9]:
#PA
#newvalue_future and _historical and sumbin_future and _historical
import xarray as xr
import numpy as np
import warnings


warnings.filterwarnings('ignore', category=UserWarning)

models = ["GAM", "GBM"]
taxas = ["Bird", "Mammals", "Amphibians"]
scenarios = ["rcp26", "rcp60"]
historical_time = 1146
future_time = 65

year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

for time in [ future_time]:
    for scenario in scenarios:
        # Create empty arrays for storing the results for each taxa separately
        diff_value_bin_combined_per_taxa = {}
        diff_sum_bin_combined_per_taxa = {}
        diff_combined_per_taxa = {}

        for taxa in taxas:
            diff_value_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_sum_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_combined_per_taxa[taxa] = np.zeros([360, 720])

        # Initialize the combined arrays outside the taxa loop
        diff_value_bin_combined = np.zeros([360, 720])
        diff_sum_bin_combined = np.zeros([360, 720])
        diff_combined = np.zeros([360, 720])

        for row, taxa in enumerate(taxas):
            # Reset the per model arrays for each taxa
            diff_value_bin_models = []
            diff_sum_bin_models = []
            diff_models = []
            percent_changes = []

            for model in models:
                newvalue_path = f"/storage/scratch/users/ch21o450/data/intermediate_results/PA_{taxa}_{model}_{time}_{scenario}_summedprobs_newvalue.nc"
                sumbin_path = f"/storage/scratch/users/ch21o450/data/intermediate_results/PA_{taxa}_{model}_{time}_{scenario}_summedprobs_sum.nc"

                mean_value_bin = xr.open_dataset(newvalue_path, decode_times=False).to_array().isel(variable=0)
                mean_sum_bin = xr.open_dataset(sumbin_path, decode_times=False).to_array().isel(variable=0)

                # Calculate the differences
                diff_value_bin = mean_value_bin
                diff_sum_bin = mean_sum_bin
                diff = diff_sum_bin - diff_value_bin
                change_percent = ((diff_sum_bin - diff_value_bin) / (diff_value_bin)) * 100
                percent_changes.append(change_percent)

                # append the differences to the model-specific lists
                diff_value_bin_models.append(diff_value_bin)
                diff_sum_bin_models.append(diff_sum_bin)
                diff_models.append(diff)

            # Calculate the ensemble mean outside of the innermost loop
            diff_value_bin_ensemble = xr.concat(diff_value_bin_models, dim="ensemble").mean(dim="ensemble")
            diff_sum_bin_ensemble = xr.concat(diff_sum_bin_models, dim="ensemble").mean(dim="ensemble")

            diff_value_bin_ensemble = diff_value_bin_ensemble
            diff_sum_bin_ensemble = diff_sum_bin_ensemble
            diff_ensemble = diff_sum_bin_ensemble - diff_value_bin_ensemble

            # Append the ensemble mean to the overall results for this taxa
            diff_value_bin_combined_per_taxa[taxa] = diff_value_bin_ensemble
            diff_sum_bin_combined_per_taxa[taxa] = diff_sum_bin_ensemble
            diff_combined_per_taxa[taxa] = diff_ensemble

        # Calculate the overall species richness sum over all taxa
        diff_value_bin_combined = sum(diff_value_bin_combined_per_taxa.values())
        diff_sum_bin_combined = sum(diff_sum_bin_combined_per_taxa.values())
        diff_combined = sum(diff_combined_per_taxa.values())

        diff_newvalue = diff_value_bin_combined
        diff_sumbin = diff_sum_bin_combined
        diff_combined = diff_combined

        # Get rid of fine line at equator
        land_sea_mask = xr.open_dataset("/storage/workspaces/wa_climate/climate_trt/chari/LUH2/remapped_luh2_ssp126.nc", decode_times=False).primf
        diff_newvalue_masked = diff_newvalue.where(land_sea_mask >= 0)
        diff_newvalue = diff_newvalue_masked.isel(time=0)

        output_path_diff_newvalue = f"/storage/scratch/users/ch21o450/data/SRnew/SR_CC_{time}_{scenario}_PA.nc"
        diff_newvalue.to_netcdf(output_path_diff_newvalue, format='NETCDF4')

        # Write out diff_sumbin as netCDF
        output_path_diff_sumbin = f"/storage/scratch/users/ch21o450/data/SRnew/SR_CCLUC_{time}_{scenario}_PA.nc"
        diff_sumbin.to_netcdf(output_path_diff_sumbin, format='NETCDF4')


FileNotFoundError: [Errno 2] No such file or directory: b'/storage/scratch/users/ch21o450/data/intermediate_results/PA_Amphibians_GAM_65_rcp26_summedprobs_newvalue.nc'

# Sanity Check


In [2]:
#Dispersal scenario
#newvalue_future and _historical and sumbin_future and _historical
import xarray as xr
import numpy as np
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

models = ["GAM", "GBM"]
taxas = ["Bird", "Mammals", "Amphibians"]
scenarios = [ "rcp60"]
#historical_time = 1146
future_time = 1146

year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

for time in [future_time]:
    for scenario in scenarios:
        # Create empty arrays for storing the results for each taxa separately
        diff_value_bin_combined_per_taxa = {}
        diff_sum_bin_combined_per_taxa = {}
        diff_combined_per_taxa = {}

        for taxa in taxas:
            diff_value_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_sum_bin_combined_per_taxa[taxa] = np.zeros([360, 720])
            diff_combined_per_taxa[taxa] = np.zeros([360, 720])

        # Initialize the combined arrays outside the taxa loop
        diff_value_bin_combined = np.zeros([360, 720])
        diff_sum_bin_combined = np.zeros([360, 720])
        diff_combined = np.zeros([360, 720])

        for row, taxa in enumerate(taxas):
            # Reset the per model arrays for each taxa
            diff_value_bin_models = []
            diff_sum_bin_models = []
            diff_models = []
            percent_changes = []

            for model in models:
                newvalue_path = f"/storage/scratch/users/ch21o450/data/LandClim_Output_sanity/{taxa}_{model}_{time}_{scenario}_summedprobs_newvalue.nc"
                sumbin_path = f"/storage/scratch/users/ch21o450/data/LandClim_Output_sanity/{taxa}_{model}_{time}_{scenario}_summedprobs_sum.nc"

                mean_value_bin = xr.open_dataset(newvalue_path, decode_times=False).to_array().isel(variable=0)
                mean_sum_bin = xr.open_dataset(sumbin_path, decode_times=False).to_array().isel(variable=0)

                # Calculate the differences
                diff_value_bin = mean_value_bin
                diff_sum_bin = mean_sum_bin
                diff = diff_sum_bin - diff_value_bin
                change_percent = ((diff_sum_bin - diff_value_bin) / (diff_value_bin)) * 100
                percent_changes.append(change_percent)

                # append the differences to the model-specific lists
                diff_value_bin_models.append(diff_value_bin)
                diff_sum_bin_models.append(diff_sum_bin)
                diff_models.append(diff)

            # Calculate the ensemble mean outside of the innermost loop
            diff_value_bin_ensemble = xr.concat(diff_value_bin_models, dim="ensemble").mean(dim="ensemble")
            diff_sum_bin_ensemble = xr.concat(diff_sum_bin_models, dim="ensemble").mean(dim="ensemble")

            diff_value_bin_ensemble = diff_value_bin_ensemble
            diff_sum_bin_ensemble = diff_sum_bin_ensemble
            diff_ensemble = diff_sum_bin_ensemble - diff_value_bin_ensemble

            # Append the ensemble mean to the overall results for this taxa
            diff_value_bin_combined_per_taxa[taxa] = diff_value_bin_ensemble
            diff_sum_bin_combined_per_taxa[taxa] = diff_sum_bin_ensemble
            diff_combined_per_taxa[taxa] = diff_ensemble

        # Calculate the overall species richness sum over all taxa
        diff_value_bin_combined = sum(diff_value_bin_combined_per_taxa.values())
        diff_sum_bin_combined = sum(diff_sum_bin_combined_per_taxa.values())
        diff_combined = sum(diff_combined_per_taxa.values())

        diff_newvalue = diff_value_bin_combined
        diff_sumbin = diff_sum_bin_combined
        diff_combined = diff_combined

        # Get rid of fine line at equator
        land_sea_mask = xr.open_dataset("/storage/workspaces/wa_climate/climate_trt/chari/LUH2/remapped_luh2_ssp126.nc", decode_times=False).primf
        diff_newvalue_masked = diff_newvalue.where(land_sea_mask >= 0)
        diff_newvalue = diff_newvalue_masked.isel(time=0)

        output_path_diff_newvalue = f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{time}_{scenario}_sanity.nc"
        diff_newvalue.to_netcdf(output_path_diff_newvalue, format='NETCDF4')

        # Write out diff_sumbin as netCDF
        output_path_diff_sumbin = f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{time}_{scenario}_sanity.nc"
        diff_sumbin.to_netcdf(output_path_diff_sumbin, format='NETCDF4')


FileNotFoundError: [Errno 2] No such file or directory: b'/storage/scratch/users/ch21o450/data/LandClim_Output_sanity/Bird_GAM_1146_rcp60_summedprobs_newvalue.nc'